## Libraries

In [1]:
from pymongo import MongoClient
import pandas as pd
import geopandas as gdp
import shapely

## Converting  Pandas Dataframe to a MongoDB Geopoint Object

### Schools collections

In [28]:
df_pets=pd.read_csv("school.csv")
df_pets.head()
df_pets.columns

Index(['name', 'latitud', 'longitud'], dtype='object')

In [29]:
gdf_pets = gdp.GeoDataFrame(df_pets, geometry= gdp.points_from_xy(df_pets.longitud, df_pets.latitud ))
gdf_pets.columns=['name', 'lat','long','loc']
gdf_pets.head()



,name,lat,long,loc
0,Gourmet Experience,40.427777,-3.687296,POINT (-3.68730 40.42778)
1,Monumento a Alfonso XII de España,40.417081,-3.683140,POINT (-3.68314 40.41708)
2,Fox In A Box Madrid - Juego de Escape Game,40.420115,-3.697489,POINT (-3.69749 40.42012)
3,Fuencarral 52 B&B,40.423500,-3.700465,POINT (-3.70047 40.42350)
4,B&B Restaurante,40.421384,-3.697403,POINT (-3.69740 40.42138)


In [30]:
gdf_pets['loc']=gdf_pets['loc'].apply(lambda x:shapely.geometry.mapping(x))
gdf_pets.head()

,name,lat,long,loc
0,Gourmet Experience,40.427777,-3.687296,"{'type': 'Point', 'coordinates': (-3.687295697..."
1,Monumento a Alfonso XII de España,40.417081,-3.683140,"{'type': 'Point', 'coordinates': (-3.683139741..."
2,Fox In A Box Madrid - Juego de Escape Game,40.420115,-3.697489,"{'type': 'Point', 'coordinates': (-3.697489005..."
3,Fuencarral 52 B&B,40.423500,-3.700465,"{'type': 'Point', 'coordinates': (-3.700465371..."
4,B&B Restaurante,40.421384,-3.697403,"{'type': 'Point', 'coordinates': (-3.697403436..."


In [31]:
client = MongoClient()
db = client.geo
collection = db.pets
collection.create_index([("loc", "2dsphere")])

'loc_2dsphere'

In [32]:
data = gdf_pets.to_dict(orient='records')
collection.insert_many(data)

### Vegan collections

In [17]:
df_vegan=pd.read_csv("vegan.csv")
df_vegan.head()

,name,latitud,longitud
0,Vinos De Bellota,40.420676,-3.679059
1,De María,40.425839,-3.697331
2,D'Mystic,40.423215,-3.698125
3,SANZ CABREJAS Bufete de Abogados penalista,40.435540,-3.686364
4,D'kebab Food,40.422261,-3.699776


In [18]:
gdf_vegan = gdp.GeoDataFrame(df_vegan, geometry= gdp.points_from_xy(df_vegan.longitud, df_vegan.latitud ))
gdf_vegan.columns=['name', 'lat','long','loc']
gdf_vegan.head()


,name,lat,long,loc
0,Vinos De Bellota,40.420676,-3.679059,POINT (-3.67906 40.42068)
1,De María,40.425839,-3.697331,POINT (-3.69733 40.42584)
2,D'Mystic,40.423215,-3.698125,POINT (-3.69813 40.42321)
3,SANZ CABREJAS Bufete de Abogados penalista,40.435540,-3.686364,POINT (-3.68636 40.43554)
4,D'kebab Food,40.422261,-3.699776,POINT (-3.69978 40.42226)


In [19]:
gdf_vegan['loc']=gdf_vegan['loc'].apply(lambda x:shapely.geometry.mapping(x))
gdf_vegan.head()

,name,lat,long,loc
0,Vinos De Bellota,40.420676,-3.679059,"{'type': 'Point', 'coordinates': (-3.679058690..."
1,De María,40.425839,-3.697331,"{'type': 'Point', 'coordinates': (-3.697330845..."
2,D'Mystic,40.423215,-3.698125,"{'type': 'Point', 'coordinates': (-3.698125448..."
3,SANZ CABREJAS Bufete de Abogados penalista,40.435540,-3.686364,"{'type': 'Point', 'coordinates': (-3.686363697..."
4,D'kebab Food,40.422261,-3.699776,"{'type': 'Point', 'coordinates': (-3.699776216..."


In [20]:
client = MongoClient()
db = client.geo
collection = db.vegan
collection.create_index([("loc", "2dsphere")])

'loc_2dsphere'

In [21]:
data_vegan = gdf_vegan.to_dict(orient='records')
collection.insert_many(data_vegan)